In [7]:
import json
import numpy as np
import os
import statistics as st
from statistics import multimode
import math
from scipy.interpolate import interp1d
import glob
import pandas as pd
import time

In [8]:
# %%time
data_folder = 'E://eyetracker/Extracts/'


In [9]:
#screen time on and off percent
def time_on_off(list1,list2):
    on_off_count = 0
    # print("len list1", len(list1))
    # print("len list2", len(list2))
    for index in range(0, len(list1)):
        # print("list1[index]", list1[index])
        # print("list2[index]", list2[index])
        if list1[index] > 0 and list2[index] > 0:
            on_off_count += 1
    # print("on_off_count", on_off_count)
    return (on_off_count/len(list1))*100

# def time_on_off(list1,list2):
#     on_off_count = list1.count(0) and list2.count(0)
#     return "{:.4f}".format((on_off_count/len(list1))*100)


#replace 0 with mean
def replace_mean(r_eye_x,r_eye_y,l_eye_x,l_eye_y):

    l_eye_x = [st.mean(l_eye_x) if (a==0) else a for a in l_eye_x]
    l_eye_y = [st.mean(l_eye_y) if (b==0) else b for b in l_eye_y]
    r_eye_x = [st.mean(r_eye_x) if (c==0) else c for c in r_eye_x]
    r_eye_y = [st.mean(r_eye_y) if (d==0) else d for d in r_eye_y]

    return (r_eye_x,r_eye_y,l_eye_x,l_eye_y)


#replace 0 with median
def replace_median(r_eye_x,r_eye_y,l_eye_x,l_eye_y):

    l_eye_x = [st.median(l_eye_x) if (a==0) else a for a in l_eye_x]
    l_eye_y = [st.median(l_eye_y) if (b==0) else b for b in l_eye_y]
    r_eye_x = [st.median(r_eye_x) if (c==0) else c for c in r_eye_x]
    r_eye_y = [st.median(r_eye_y) if (d==0) else d for d in r_eye_y]

    return (r_eye_x,r_eye_y,l_eye_x,l_eye_y)


#replace 0 with second mode as first mode always 0
def replace_mode(r_eye_x,r_eye_y,l_eye_x,l_eye_y):

    l_eye_x_temp = [a for a in l_eye_x if a != 0]
    l_eye_y_temp = [b for b in l_eye_y if b != 0]
    r_eye_x_temp = [c for c in r_eye_x if c != 0]
    r_eye_y_temp = [d for d in r_eye_y if d != 0]

    l_eye_x = [st.mode(l_eye_x_temp) if (a==0) else a for a in l_eye_x]
    l_eye_y = [st.mode(l_eye_y_temp) if (b==0) else b for b in l_eye_y]
    r_eye_x = [st.mode(r_eye_x_temp) if (c==0) else c for c in r_eye_x]
    r_eye_y = [st.mode(r_eye_y_temp) if (d==0) else d for d in r_eye_y]

    return (r_eye_x,r_eye_y,l_eye_x,l_eye_y)


#replace 0 using interpolation
def replace_interpol(r_eye_x,r_eye_y,l_eye_x,l_eye_y):

    #first replace extreme values with meadin values
    if r_eye_x[-1]==0.0: r_eye_x[-1] = st.median(r_eye_x)
    if r_eye_y[-1]==0.0: r_eye_y[-1] = st.median(r_eye_y)
    if l_eye_x[-1]==0.0: l_eye_x[-1] = st.median(l_eye_x)
    if l_eye_y[-1]==0.0: l_eye_y[-1] = st.median(l_eye_y)

    if r_eye_x[0]==0.0: r_eye_x[0] = st.median(r_eye_x)
    if r_eye_y[0]==0.0: r_eye_y[0] = st.median(r_eye_y)
    if l_eye_x[0]==0.0: l_eye_x[0] = st.median(l_eye_x)
    if l_eye_y[0]==0.0: l_eye_y[0] = st.median(l_eye_y)

    r_eye_x = interpolation(r_eye_x)
    r_eye_y = interpolation(r_eye_y)
    l_eye_x = interpolation(l_eye_x)
    l_eye_y = interpolation(l_eye_y)

    return (r_eye_x,r_eye_y,l_eye_x,l_eye_y)


#replace 0 using interpolation
def interpolation(list):

    y = np.array(list)
    x = np.arange(len(y))
    idx = np.where(y!=0)
    f = interp1d(x[idx],y[idx],bounds_error=False)
    y_new = f(x)

    return y_new


#calculate distance for every point
def calculate_velocity(r_eye_x,r_eye_y,l_eye_x,l_eye_y):
    r_eye_vel = []
    l_eye_vel = []

    for i in range (0,len(r_eye_x)-1):
        dist_r = ( (r_eye_x[i+1] - r_eye_x[i])**2 + (r_eye_y[i+1] - r_eye_y[i])**2 ) **0.5
        r_eye_vel.append(dist_r/(len(r_eye_x)))

    for i in range (0,len(l_eye_x)-1):
        dist_l = ( (l_eye_x[i+1] - l_eye_x[i])**2 + (l_eye_y[i+1] - l_eye_y[i])**2 ) **0.5
        l_eye_vel.append(dist_l/(len(l_eye_x)))

    mean_vel = (st.mean(r_eye_vel) + st.mean(l_eye_vel)) / 2

    min_vel = (min(r_eye_vel) + min(l_eye_vel)) / 2
    max_vel = (max(r_eye_vel) + max(l_eye_vel)) / 2

    return "{:.4f}".format(mean_vel),"{:.4f}".format(min_vel),"{:.4f}".format(max_vel)

In [10]:
# %%time

data_folder = 'E://eyetracker/Extracts/'
data_metrics1 = [[],[],[],[],[]]
data_metrics2 = [[],[],[],[],[]]
data_metrics3 = [[],[],[],[],[]]
data_metrics4 = [[],[],[],[],[]]
data_metrics5 = [[],[],[],[],[]]

data_metrics = [data_metrics1, data_metrics2, data_metrics3,
                data_metrics4, data_metrics5]

total_files = int(len(glob.glob(data_folder + '*')))
file_count = 0
#read all files from unzipped folder
skipped_files = []
for file in glob.glob(data_folder + '*'):
    if file_count % 50 == 0:
        print(file_count, " Completed")
    file_count += 1
    # if file_count == 15:
    #     break
    head_tail = os.path.split(file)
    #this will give sample id
    tail = head_tail[1]

    r_eye_x = []
    r_eye_y = []
    l_eye_x = []
    l_eye_y = []
    face_x = []
    face_y = []

    with open(data_folder + tail + '/appleLeftEye.json', 'r') as json_file_l_eye:
        json_load_l_eye = json.load(json_file_l_eye)

        for item in json_load_l_eye['X']:
            l_eye_x.append(item)

        for item in json_load_l_eye['Y']:
            l_eye_y.append(item)

    with open(data_folder + tail + '/appleRightEye.json', 'r') as json_file_r_eye:
        json_load_r_eye = json.load(json_file_r_eye)

        for item in json_load_r_eye['X']:
            r_eye_x.append(item)

        for item in json_load_r_eye['Y']:
            r_eye_y.append(item)

    with open(data_folder + tail + '/appleFace.json', 'r') as json_file_face:
        json_load_face = json.load(json_file_face)

        for item in json_load_face['X']:
            face_x.append(item)

        for item in json_load_face['Y']:
            face_y.append(item)

    #uncomment following if you want to impute 0 value with mean,median,mode or interpolation

    for i in range (0,len(data_metrics)):
        try:
            screen_time_percent_r = time_on_off(r_eye_x,r_eye_y)
            screen_time_percent_l = time_on_off(l_eye_x,l_eye_y)
            screen_time_percent = (screen_time_percent_r + screen_time_percent_l) / 2
            # print("Screen_r", screen_time_percent_r)
            # print("Screen_l", screen_time_percent_l)
            # print("Screen", screen_time_percent)
            screen_time_percent = "{:.4f}".format(screen_time_percent)

            if i == 0:
                r_eye_x,r_eye_y,l_eye_x,l_eye_y = replace_mean(r_eye_x,r_eye_y,l_eye_x,l_eye_y)
            elif i == 1:
                r_eye_x,r_eye_y,l_eye_x,l_eye_y = replace_median(r_eye_x,r_eye_y,l_eye_x,l_eye_y)
            elif i == 2:
                r_eye_x,r_eye_y,l_eye_x,l_eye_y = replace_mode(r_eye_x,r_eye_y,l_eye_x,l_eye_y)
            elif i == 3:
                r_eye_x,r_eye_y,l_eye_x,l_eye_y =replace_interpol(r_eye_x,r_eye_y,l_eye_x,l_eye_y)

            mean_vel,min_vel,max_vel = calculate_velocity(r_eye_x,r_eye_y,l_eye_x,l_eye_y)
            #screen_time_percent = time_on_off(face_x,face_y)

        except BaseException as err:
            print(f"Unexpected {err=}, {type(err)=}")
            skipped_files.append(tail)
            print("Had to skip", tail)
            continue
        data_metrics[i][0].append(tail)
        data_metrics[i][1].append(mean_vel)
        data_metrics[i][2].append(min_vel)
        data_metrics[i][3].append(max_vel)
        data_metrics[i][4].append(screen_time_percent)
print("Done")

0  Completed
50  Completed
100  Completed
150  Completed
200  Completed
250  Completed
300  Completed
350  Completed
400  Completed
450  Completed
500  Completed
550  Completed
Unexpected err=StatisticsError('no mode for empty data'), type(err)=<class 'statistics.StatisticsError'>
Had to skip 01185
Unexpected err=ValueError('cannot reshape array of size 0 into shape (0,newaxis)'), type(err)=<class 'ValueError'>
Had to skip 01185
600  Completed
650  Completed
700  Completed
750  Completed
800  Completed
Unexpected err=StatisticsError('no mode for empty data'), type(err)=<class 'statistics.StatisticsError'>
Had to skip 01730
Unexpected err=ValueError('cannot reshape array of size 0 into shape (0,newaxis)'), type(err)=<class 'ValueError'>
Had to skip 01730
850  Completed
900  Completed
950  Completed
Unexpected err=StatisticsError('no mode for empty data'), type(err)=<class 'statistics.StatisticsError'>
Had to skip 02065
Unexpected err=ValueError('cannot reshape array of size 0 into shape

In [11]:
for i in range (0,len(data_metrics)):
    df = pd.DataFrame({'Sample':data_metrics[i][0],'Mean_vel':data_metrics[i][1],
                  'Min_vel':data_metrics[i][2],'Max_vel':data_metrics[i][3],
                  'Screen_time_off_percent':data_metrics[i][4]})
    df.to_csv(data_folder + 'replaced_with_type' + str(i) + '_v2.csv',index=False)
#df

In [12]:
print("Skipped:", skipped_files)

Skipped: ['01185', '01185', '01730', '01730', '02065', '02065']
